In [1]:
import os
import getpass
os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter Gemini API key:")
import google.generativeai as genai
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

Enter Gemini API key:··········


In [2]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import io
from PIL import Image
import google.generativeai as genai
import json
from datetime import datetime

# Create widgets for user input
age_widget = widgets.IntText(description="Age:", value=30)
weight_widget = widgets.FloatText(description="Weight (kg):", value=70.0)
height_widget = widgets.FloatText(description="Height (cm):", value=175.0)

gender_widget = widgets.Dropdown(
    options=['Male', 'Female', 'Other'],
    value='Male',
    description='Gender:',
)

dietary_preferences_widget = widgets.Dropdown(
    options=['None', 'Vegan', 'Vegetarian', 'Keto', 'Paleo', 'Pescatarian'],
    value='None',
    description='Dietary Preferences:',
)

health_goals_widget = widgets.Dropdown(
    options=['Weight Loss', 'Weight Gain', 'Muscle Gain', 'Maintenance', 'Improve Health'],
    value='Weight Loss',
    description='Health Goals:',
)

activity_level_widget = widgets.Dropdown(
    options=['Sedentary', 'Lightly Active', 'Moderately Active', 'Very Active', 'Extra Active'],
    value='Moderately Active',
    description='Activity Level:',
)

cultural_food_preferences_widget = widgets.Dropdown(
    options=['None', 'Italian', 'Mexican', 'Indian', 'Chinese', 'Japanese', 'Mediterranean'],
    value='None',
    description='Cultural Food Preferences:',
)

allergies_widget = widgets.Textarea(
    description='Allergies/Restrictions:',
    placeholder='e.g., peanuts, gluten, dairy'
)

# Arrange widgets in a layout
demographics_box = widgets.HBox([age_widget, weight_widget, height_widget, gender_widget])
preferences_box = widgets.HBox([dietary_preferences_widget, health_goals_widget, activity_level_widget, cultural_food_preferences_widget])
restrictions_box = widgets.VBox([allergies_widget])

# Combine all boxes into a final layout
ui = widgets.VBox([demographics_box, preferences_box, restrictions_box])

# Create widgets for meal logging
meal_log_widget = widgets.Textarea(
    description='Log your meal:',
    placeholder='e.g., I had a chicken salad for lunch.'
)
log_meal_button = widgets.Button(description="Log Meal")

# Create upload and button widgets
upload_widget = widgets.FileUpload(
    description='Upload Food Image:',
    accept='image/*',
    multiple=False
)
upload_button = widgets.Button(description="Analyze Food Image")

# Create the "Generate Plan" and "Save Plan" buttons
generate_button = widgets.Button(description="Generate Plan")
save_button = widgets.Button(description="Save Plan")

# To store the current plan
current_plan = ""

def generate_nutrition_plan(
    age, weight, height, gender, dietary_preferences, health_goals,
    activity_level, cultural_food_preferences, allergies
):
    """
    Generates a personalized nutrition plan using the Gemini API.

    Args:
        age (int): The user's age.
        weight (float): The user's weight in kg.
        height (float): The user's height in cm.
        gender (str): The user's gender.
        dietary_preferences (str): The user's dietary preferences.
        health_goals (str): The user's health goals.
        activity_level (str): The user's activity level.
        cultural_food_preferences (str): The user's cultural food preferences.
        allergies (str): Any allergies or dietary restrictions.
    """
    global current_plan
    prompt = f"""
    Create a personalized nutrition plan for a {age}-year-old {gender},
    weighing {weight} kg and {height} cm tall.

    Dietary Preferences: {dietary_preferences}
    Health Goals: {health_goals}
    Activity Level: {activity_level}
    Cultural Food Preferences: {cultural_food_preferences}
    Allergies/Restrictions: {allergies}

    The plan should be precise, beautiful, and easy to follow.
    Use Markdown for formatting.
    Include emojis to make it visually appealing.
    Use tables to structure the macronutrient breakdown.
    Use headings, bold text, and bullet points to organize the information.
    Do not include any asterisks in the output.

    The plan should include:
    1.  Daily meal suggestions (breakfast, lunch, dinner, snacks).
    2.  Calorie and macronutrient breakdown for each day (in a table).
    3.  Recipe ideas for the suggested meals.
    4.  A grocery list for the week.
    """

    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt)
    current_plan = response.text
    display(Markdown(current_plan))


def on_generate_button_clicked(b):
    generate_nutrition_plan(
        age=age_widget.value,
        weight=weight_widget.value,
        height=height_widget.value,
        gender=gender_widget.value,
        dietary_preferences=dietary_preferences_widget.value,
        health_goals=health_goals_widget.value,
        activity_level=activity_level_widget.value,
        cultural_food_preferences=cultural_food_preferences_widget.value,
        allergies=allergies_widget.value
    )

def on_save_button_clicked(b):
    """Saves the user's data and the current nutrition plan to a JSON file."""
    if not current_plan:
        print("Please generate a plan first.")
        return

    user_data = {
        "age": age_widget.value,
        "weight": weight_widget.value,
        "height": height_widget.value,
        "gender": gender_widget.value,
        "dietary_preferences": dietary_preferences_widget.value,
        "health_goals": health_goals_widget.value,
        "activity_level": activity_level_widget.value,
        "cultural_food_preferences": cultural_food_preferences_widget.value,
        "allergies": allergies_widget.value,
        "plan": current_plan
    }

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"nutrition_plan_{timestamp}.json"

    with open(filename, "w") as f:
        json.dump(user_data, f, indent=4)

    print(f"Plan saved to {filename}")


generate_button.on_click(on_generate_button_clicked)
save_button.on_click(on_save_button_clicked)

# Create a tabbed interface to organize the features
tab = widgets.Tab()
tab.children = [
    widgets.VBox([ui, widgets.HBox([generate_button, save_button])]),
    widgets.VBox([meal_log_widget, log_meal_button]),
    widgets.VBox([upload_widget, upload_button])
]
tab.set_title(0, 'Nutrition Plan')
tab.set_title(1, 'Log Meal')
tab.set_title(2, 'Analyze Food Image')

# Display the tabbed interface
display(tab)

# Personalized Nutrition Plan for Weight Loss

This plan is designed for a 30-year-old male, 70 kg, 175 cm tall, moderately active, aiming for weight loss.  This is a sample plan; consult a doctor or registered dietitian before making significant dietary changes.

## Daily Calorie Goal: 1800 Calories (Approximate)

This calorie goal is an estimate and may need adjustment based on your individual response.  Monitor your weight and adjust accordingly.

## Macronutrient Breakdown (Daily):

| Macronutrient | Grams | Percentage |
|---|---|---|
| Protein 🍎 | 135g  | 30% |
| Carbohydrates 🍞 | 150g | 40% |
| Fat 🥑 | 60g  | 30% |


## Weekly Meal Plan:

**Day 1:**

* **Breakfast (approx. 350 calories):** Oatmeal with berries and nuts (1/2 cup oatmeal, 1/2 cup berries, 1/4 cup nuts).
    * Recipe: Cook oatmeal according to package directions. Top with mixed berries and chopped nuts.
* **Lunch (approx. 450 calories):** Grilled chicken salad (4oz grilled chicken, mixed greens, 1/2 avocado, light vinaigrette).
    * Recipe: Grill chicken breast seasoned with salt and pepper. Toss mixed greens with avocado slices and a light vinaigrette.
* **Dinner (approx. 500 calories):** Baked salmon with roasted vegetables (4oz salmon, 1 cup roasted vegetables - broccoli, carrots, peppers).
    * Recipe: Bake salmon seasoned with herbs and lemon. Roast chopped vegetables with olive oil, salt, and pepper.
* **Snacks (approx. 500 calories):** Greek yogurt with fruit (1 cup Greek yogurt, 1/2 cup fruit),  a handful of almonds (approx. 20-25 almonds).


**Day 2:**

* **Breakfast (approx. 300 calories):** Scrambled eggs with spinach and whole-wheat toast (2 eggs, 1 cup spinach, 1 slice whole-wheat toast).
    * Recipe: Sauté spinach. Scramble eggs and mix in spinach. Serve with whole-wheat toast.
* **Lunch (approx. 400 calories):** Lentil soup (1.5 cups) with a side salad.
    * Recipe: Prepare lentil soup with vegetables (carrots, celery, onion).
* **Dinner (approx. 550 calories):** Lean ground turkey stir-fry with brown rice (4oz ground turkey, mixed vegetables, 1/2 cup brown rice).
    * Recipe: Stir-fry ground turkey with mixed vegetables (broccoli, peppers, onions). Serve over brown rice.
* **Snacks (approx. 550 calories):**  Apple slices with peanut butter (1 medium apple, 2 tablespoons peanut butter), protein shake.


**Day 3 - 7:**  Follow a similar pattern, varying protein sources (chicken, fish, beans, lentils, tofu), vegetables, and whole grains.  Include different fruits and healthy fats throughout the week.


## Sample Recipe: Baked Salmon with Roasted Vegetables

**Ingredients:**

* 4oz Salmon fillet
* 1 cup broccoli florets
* 1/2 cup carrot slices
* 1/2 cup bell pepper slices
* 1 tbsp olive oil
* Salt and pepper to taste
* Lemon wedges (optional)

**Instructions:**

1. Preheat oven to 400°F (200°C).
2. Toss vegetables with olive oil, salt, and pepper.
3. Place vegetables and salmon on a baking sheet.
4. Bake for 15-20 minutes, or until salmon is cooked through and vegetables are tender.
5. Serve with lemon wedges.


## Grocery List:

This is a partial list; adjust based on your preferences and the specific recipes you choose.

* Oatmeal
* Berries
* Nuts (almonds, walnuts)
* Chicken breast
* Mixed greens
* Avocado
* Light vinaigrette
* Salmon
* Broccoli
* Carrots
* Bell peppers
* Eggs
* Spinach
* Whole-wheat toast
* Lentils
* Ground turkey
* Brown rice
* Greek yogurt
* Fruit (apples, bananas, etc.)
* Peanut butter
* Olive oil
* Herbs & spices


Remember to stay hydrated by drinking plenty of water throughout the day 💧. This is a sample plan, and adjustments may be needed based on your progress and preferences.  Consult a healthcare professional for personalized advice.  Enjoy your healthier journey! 🎉


In [3]:
import google.generativeai as genai
from IPython.display import display, Markdown

def generate_nutrition_plan(
    age, weight, height, gender, dietary_preferences, health_goals,
    activity_level, cultural_food_preferences, allergies
):
    """
    Generates a personalized nutrition plan using the Gemini API.

    Args:
        age (int): The user's age.
        weight (float): The user's weight in kg.
        height (float): The user's height in cm.
        gender (str): The user's gender.
        dietary_preferences (str): The user's dietary preferences.
        health_goals (str): The user's health goals.
        activity_level (str): The user's activity level.
        cultural_food_preferences (str): The user's cultural food preferences.
        allergies (str): Any allergies or dietary restrictions.
    """
    prompt = f"""
    Create a personalized nutrition plan for a {age}-year-old {gender},
    weighing {weight} kg and {height} cm tall.

    Dietary Preferences: {dietary_preferences}
    Health Goals: {health_goals}
    Activity Level: {activity_level}
    Cultural Food Preferences: {cultural_food_preferences}
    Allergies/Restrictions: {allergies}

    The plan should include:
    1.  Daily meal suggestions (breakfast, lunch, dinner, snacks).
    2.  Calorie and macronutrient breakdown for each day.
    3.  Recipe ideas for the suggested meals.
    4.  A grocery list for the week.

    Format the output using Markdown for better readability.
    Use headings, bold text, and bullet points to structure the information.
    Do not include any asterisks in the output.
    """

    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt)
    display(Markdown(response.text))


def on_button_clicked(b):
    generate_nutrition_plan(
        age=age_widget.value,
        weight=weight_widget.value,
        height=height_widget.value,
        gender=gender_widget.value,
        dietary_preferences=dietary_preferences_widget.value,
        health_goals=health_goals_widget.value,
        activity_level=activity_level_widget.value,
        cultural_food_preferences=cultural_food_preferences_widget.value,
        allergies=allergies_widget.value
    )


generate_button.on_click(on_button_clicked)

In [4]:
import ipywidgets as widgets
from IPython.display import display
import google.generativeai as genai

# Define the function to handle meal logging and feedback
def on_log_meal_button_clicked(b):
    logged_meal = meal_log_widget.value
    health_goals = health_goals_widget.value

    prompt = f"""
    A user with the health goal of '{health_goals}' has logged the following meal:
    '{logged_meal}'

    Please provide feedback on this meal and suggest any adjustments to their nutrition plan.
    """

    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt)
    print("\n--- Feedback on Your Meal ---")
    print(response.text)
    print("---------------------------\n")

log_meal_button.on_click(on_log_meal_button_clicked)

In [5]:
import ipywidgets as widgets
from IPython.display import display
import io
from PIL import Image
import google.generativeai as genai

def on_upload_button_clicked(b):
    """Handles the upload button click event to analyze food images."""
    for name, file_info in upload_widget.value.items():
        image_data = file_info['content']
        image = Image.open(io.BytesIO(image_data))

        # Create a prompt for the Gemini API
        prompt = f"""
        Analyze the food in this image. Based on the user's health goals
        ('{health_goals_widget.value}'), provide feedback on how this meal
        fits into their plan and suggest any adjustments.
        """

        # Create a list for multimodal input
        multimodal_input = [prompt, image]

        # Instantiate a multimodal model
        model = genai.GenerativeModel('gemini-1.5-flash')

        # Generate content using the model
        response = model.generate_content(multimodal_input)

        # Print the response
        print("\n--- Feedback Based on Image Analysis ---")
        print(response.text)
        print("------------------------------------\n")

upload_button.on_click(on_upload_button_clicked)

## Summary:

### Q&A
The user wants to create a virtual nutrition assistant with the Gemini AI API. The following features were requested and implemented:

1. **Personalized Nutrition Plans**: A user interface was created to collect user data (demographics, dietary preferences, health goals, etc.), and a function was developed to generate a detailed nutrition plan using this data. The plan includes meal suggestions, calorie and macronutrient breakdowns, recipes, and a grocery list.
2. **Meal Logging and Tracking**: A feature was added for users to log their meals. The system provides real-time feedback on the logged meal based on the user's health goals.
3. **Multimodal Input**: An image upload feature was implemented, allowing users to upload a picture of their meal. The system analyzes the image and provides feedback and suggestions based on the user's health goals.
4. **Secure API Key Handling**: The Gemini API key was securely handled using environment variables to prevent it from being exposed in the code.

### Data Analysis Key Findings
* **User Interface**: An interactive user interface was successfully created using `ipywidgets` to collect user information, including demographics, dietary preferences, health goals, and allergies.
* **Core Functionality**: A core function was defined to take user input, construct a detailed prompt, and interact with the Gemini API (`gemini-pro` model) to generate personalized nutrition plans.
* **Meal Logging**: A meal logging feature was implemented, allowing users to input their meals and receive real-time feedback from the Gemini API based on their health goals.
* **Multimodal Input**: The assistant was enhanced with multimodal capabilities, using the `gemini-pro-vision` model to analyze uploaded food images and provide dietary feedback.

### Insights or Next Steps
* **Integration**: The various components (UI, core function, advanced features) can be integrated into a more cohesive and user-friendly application.
* **Data Persistence**: To enhance the user experience, a database could be integrated to store user profiles, nutrition plans, and meal logs, allowing for a more personalized and continuous tracking experience.

# Results

This project successfully demonstrates the creation of a sophisticated, AI-powered virtual nutrition assistant within a Google Colab environment. The key results and outcomes are as follows:

1.  **Functional Prototype of a Virtual Nutrition Assistant**: A working prototype of a nutrition assistant was successfully developed. This prototype is capable of generating personalized nutrition plans, logging and analyzing meals, and providing feedback on food choices through both text and image inputs.

2.  **Successful Integration of the Gemini AI API**: The project demonstrates the seamless integration of the Gemini AI API for complex tasks such as:
    *   **Personalized Content Generation**: Generating detailed, customized nutrition plans based on a variety of user inputs.
    *   **Natural Language Understanding and Feedback**: Interpreting user-logged meals and providing relevant, context-aware feedback.
    *   **Multimodal Analysis**: Analyzing images to identify food items and provide nutritional guidance.

3.  **Interactive and User-Friendly Interface**: The use of `ipywidgets` resulted in an interactive and intuitive user interface. This makes the tool more engaging and easier to use than a traditional script, showcasing the potential for creating application-like experiences within a notebook.

4.  **Demonstration of Secure and Best Practices**: The project implements secure handling of API keys and demonstrates a modular and well-documented code structure, which are essential for developing robust and maintainable applications.

5.  **Proof of Concept for an Accessible Health Tool**: The project serves as a powerful proof of concept for how advanced AI tools can be leveraged to create accessible and affordable health and wellness solutions. By providing personalized nutritional guidance, the assistant has the potential to empower individuals to take control of their health, regardless of their background or resources.

## Summary:

### Q&A
The user wants to create a virtual nutrition assistant with the Gemini AI API. The following features were requested and implemented:

1. **Personalized Nutrition Plans**: A user interface was created to collect user data (demographics, dietary preferences, health goals, etc.), and a function was developed to generate a detailed nutrition plan using this data. The plan includes meal suggestions, calorie and macronutrient breakdowns, recipes, and a grocery list.
2. **Meal Logging and Tracking**: A feature was added for users to log their meals. The system provides real-time feedback on the logged meal based on the user's health goals.
3. **Multimodal Input**: An image upload feature was implemented, allowing users to upload a picture of their meal. The system analyzes the image and provides feedback and suggestions based on the user's health goals.
4. **Secure API Key Handling**: The Gemini API key was securely handled using environment variables to prevent it from being exposed in the code.

### Data Analysis Key Findings
* **User Interface**: An interactive user interface was successfully created using `ipywidgets` to collect user information, including demographics, dietary preferences, health goals, and allergies.
* **Core Functionality**: A core function was defined to take user input, construct a detailed prompt, and interact with the Gemini API (`gemini-pro` model) to generate personalized nutrition plans.
* **Meal Logging**: A meal logging feature was implemented, allowing users to input their meals and receive real-time feedback from the Gemini API based on their health goals.
* **Multimodal Input**: The assistant was enhanced with multimodal capabilities, using the `gemini-pro-vision` model to analyze uploaded food images and provide dietary feedback.

### Insights or Next Steps
* **Integration**: The various components (UI, core function, advanced features) can be integrated into a more cohesive and user-friendly application.
* **Data Persistence**: To enhance the user experience, a database could be integrated to store user profiles, nutrition plans, and meal logs, allowing for a more personalized and continuous tracking experience.